In [1]:
import os
import re
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import QuantileRegressor, LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_pinball_loss

In [2]:
# CONFIG
LOAD_TAG = "EWS102/10DAS0O:AI1O013305.PNT"
MAX_LOAD = 350
LOAD_MIN_PCT = 0.30
LOAD_MAX_PCT = 1.00
FLAT_WINDOW = 12
FLAT_EPS = 1e-6
OUTLIER_IQR_K = 1.5

PARQUET_DIR = "../data_parquet_per_day/tja1/"
GROUP_CONFIG = "../config/tja1.json"
PLOT_ROOT = "../plots_html/UNIT1"

START = "2025-12-01"
END   = "2025-12-16"

In [3]:
BLOCKED_TAGS = {
    "tag a",
    "tag b"
}


In [4]:
os.makedirs(PLOT_ROOT, exist_ok=True)

In [ ]:
# CSV + FORMULAS ROWS
CSV_ROWS = []
FORMULAS_ROWS = []

def add_csv_row(rule_name, tag, type_, f_pair, formula=None, target_name=None, rule_symbol=None, display_name=None):
    """
    Menambahkan row ke CSV_ROWS dan FORMULAS_ROWS dengan tambahan kolom display_name
    """
    CSV_ROWS.append({
        "rule_name": rule_name,
        "tags": tag,
        "type": type_,
        "f_pair": f_pair
    })
    
    if formula is not None and target_name is not None:
        FORMULAS_ROWS.append({
            "formula": formula,
            "tag_name": target_name,       
            "display_name": display_name,
            "area_name": rule_name,
            "simbol": rule_symbol if rule_symbol is not None else ""
        })

# UTILITIES

In [6]:
def safe_filename(name, max_len=255):
    name = re.sub(r'[\\/*?:"<>|]', "_", str(name))
    name = re.sub(r'\s+', "_", name)
    return name[:max_len]

In [7]:
def build_target_expression(info):
    cfg = info.get("target", {})

    def normalize_tags(tags):
        if isinstance(tags, dict):
            return [t for v in tags.values() for t in v]
        return tags if isinstance(tags, list) else []

    def avg_expr(tags):
        tags = normalize_tags(tags)
        if not tags:
            return ""
        if len(tags) == 1:
            return tags[0]
        return "(" + " + ".join(tags) + f")/{len(tags)}"

    # ---------- LMTD ----------
    if info["name"].upper() == "LMTD":
        avg_cfg = cfg.get("avg", {})

        def avg_or_single(tags):
            if isinstance(tags, dict):
                tags = [t for v in tags.values() for t in v]
            if not tags:
                return "0"
            if len(tags) == 1:
                return tags[0]
            return f"({' + '.join(tags)})/{len(tags)}"

        Th_in  = avg_or_single(avg_cfg.get("LMTD_Avg1", []))
        Tc_out = avg_or_single(avg_cfg.get("LMTD_Avg2", []))
        Th_out = avg_or_single(avg_cfg.get("LMTD_Avg3", []))
        Tc_in  = avg_or_single(avg_cfg.get("LMTD_Avg4", []))

        DT1 = f"({Th_in} - {Tc_out})"
        DT2 = f"({Th_out} - {Tc_in})"

        return f"({DT1} - {DT2}) / ln({DT1} / {DT2})"

    # ---------- SINGLE ----------
    if "single" in cfg and cfg["single"]:
        return cfg["single"][0]

    # ---------- DELTA ----------
    if "delta" in cfg:
        left_cfg  = cfg["delta"].get("left", {})
        right_cfg = cfg["delta"].get("right", {})

        def side_expr(side):
            tags = normalize_tags(side.get("tags", []))
            if not tags:
                return ""
            if side.get("mode") == "AVG":
                return avg_expr(tags)
            return tags[0]

        return f"({side_expr(left_cfg)} - {side_expr(right_cfg)})"

    # ---------- AVG ONLY ----------
    if "avg" in cfg and cfg["avg"]:
        tags = []
        for v in cfg["avg"].values():
            tags.extend(v)
        return avg_expr(tags)

    return ""


In [8]:
def filter_by_load(df, load_tag, max_load, pct_min, pct_max):
    if load_tag not in df.columns:
        return df
    return df[
        (df[load_tag] >= max_load * pct_min) &
        (df[load_tag] <= max_load * pct_max)
    ]

In [9]:
def detect_flatline(series, window=12, eps=1e-6):
    """
    Flatline jika range dalam window sangat kecil
    """
    return (
        series
        .rolling(window)
        .apply(lambda x: np.nanmax(x) - np.nanmin(x) < eps, raw=True)
        .fillna(False)
        .astype(bool)
    )

In [10]:
def detect_outlier_iqr(series, k=1.5):
    """
    Outlier berbasis IQR (robust)
    """
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1

    if iqr == 0 or pd.isna(iqr):
        return pd.Series(False, index=series.index)

    return (series < q1 - k * iqr) | (series > q3 + k * iqr)

In [11]:
with open(GROUP_CONFIG, "r") as f:
    groups = json.load(f)

In [12]:
# ==============================
# STRICT ENGINEER FEATURE TAGS
# ==============================
ENGINEER_FEATURE_TAGS = set()

for group_items in groups.values():
    for item in group_items:
        feat_cfg = item.get("features", {})

        # feature single
        ENGINEER_FEATURE_TAGS.update(feat_cfg.get("single", []))

        # feature avg
        for tags in feat_cfg.get("avg", {}).values():
            ENGINEER_FEATURE_TAGS.update(tags)

print(f"[INFO] ENGINEER_FEATURE_TAGS = {len(ENGINEER_FEATURE_TAGS)}")


[INFO] ENGINEER_FEATURE_TAGS = 56


In [13]:
print("CT tags in engineer features:")
for t in ENGINEER_FEATURE_TAGS:
    if "CT" in t:
        print(" -", t)

CT tags in engineer features:


In [14]:
# LOAD DATA
dfs = []
for f in os.listdir(PARQUET_DIR):
    if f.endswith(".parquet"):
        dfs.append(pd.read_parquet(os.path.join(PARQUET_DIR, f)))

df = pd.concat(dfs, ignore_index=True)

In [15]:
df = df.rename(columns={
    "f_date_rec": "timestamp",
    "f_address_no": "tag",
    "f_value": "value"
})

In [16]:
df["timestamp"] = pd.to_datetime(df["timestamp"])
df["value"] = pd.to_numeric(df["value"], errors="coerce")

In [17]:
df = df[(df["timestamp"] >= START) & (df["timestamp"] < END)]

In [18]:
df_pivot = (
    df.pivot_table(
        index="timestamp",
        columns="tag",
        values="value",
        aggfunc="mean"
    )
    .resample("1T")
    .mean()
    .interpolate(method="time")
)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_16312\4125327728.py:8: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  .resample("1T")


In [19]:
# =====================================================
# SIMPLE TAG DATA CHECK (DISPLAY ONLY)
# =====================================================

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

tag_status = []

for col in df_pivot.columns:
    if col == LOAD_TAG:
        tag_status.append({
            "tag": col,
            "status": "LOAD"
        })
        continue

    non_na_count = df_pivot[col].notna().sum()

    tag_status.append({
        "tag": col,
        "status": "OK" if non_na_count > 0 else "NO_DATA",
        "non_na_count": int(non_na_count)
    })

df_tag_status = pd.DataFrame(tag_status)

print("\n================ TAG DATA AVAILABILITY ================")
print(df_tag_status.sort_values(["status", "tag"]))

# ------------------------------
# LIST TAG TANPA DATA
# ------------------------------
missing_tags = df_tag_status.loc[
    df_tag_status["status"] == "NO_DATA", "tag"
].tolist()

if missing_tags:
    print("\n[INFO] TAG TANPA DATA:")
    for t in missing_tags:
        print(" -", t)
else:
    print("\n[INFO] Semua tag memiliki data")

print("======================================================")



================ TAG DATA AVAILABILITY ================
                                 tag status  non_na_count
550    EWS102/10DAS0O:AI1O013305.PNT   LOAD           NaN
0                               AUTO     OK       21600.0
1               AphOutletTemperature     OK       21600.0
2                  COMBUSTION ENABLE     OK       21600.0
3            EWS102/10AMILL:1PID.OUT     OK       21600.0
4            EWS102/10AMILL:2PID.OUT     OK       21600.0
5            EWS102/10BMILL:1PID.OUT     OK       21600.0
6            EWS102/10BMILL:2PID.OUT     OK       21600.0
7              EWS102/10CH:14433.CIN     OK       21600.0
8              EWS102/10CH:14434.CIN     OK       21600.0
9              EWS102/10CH:14435.CIN     OK       21600.0
10             EWS102/10CH:14436.CIN     OK       21600.0
11             EWS102/10CH:14437.CIN     OK       21600.0
12             EWS102/10CH:14438.CIN     OK       21600.0
13             EWS102/10CH:14439.CIN     OK       21600.0
14             

In [20]:
# FILTER BY LOAD
df_pivot = filter_by_load(
    df_pivot,
    LOAD_TAG,
    MAX_LOAD,
    LOAD_MIN_PCT,
    LOAD_MAX_PCT
)

# DETECT FLAT & OUTLIER
feature_cols = [
    c for c in df_pivot.columns
    if c != LOAD_TAG and df_pivot[c].dtype != "O"
]

flat_flags = []
outlier_flags = []

for col in feature_cols:
    f_flat = detect_flatline(df_pivot[col], FLAT_WINDOW, FLAT_EPS)
    f_out  = detect_outlier_iqr(df_pivot[col], OUTLIER_IQR_K)

    df_pivot[f"FLAG_FLAT_{col}"]    = f_flat
    df_pivot[f"FLAG_OUTLIER_{col}"] = f_out

    flat_flags.append(f_flat)
    outlier_flags.append(f_out)

# FLAG GLOBAL
df_pivot["FLAG_FLATLINE"] = pd.concat(flat_flags, axis=1).any(axis=1)
df_pivot["FLAG_OUTLIER"]  = pd.concat(outlier_flags, axis=1).any(axis=1)

# AUDIT COPY (INI YANG SEBELUMNYA KURANG)
df_audit = df_pivot.copy()

print("[INFO] Data quality summary (before filtering):")
print(" - Flatline rows :", int(df_audit["FLAG_FLATLINE"].sum()))
print(" - Outlier rows  :", int(df_audit["FLAG_OUTLIER"].sum()))
print(" - Total rows    :", len(df_audit))

# FILTER DATA UNTUK MODEL
df_model = df_audit[
    (~df_audit["FLAG_FLATLINE"]) &
    (~df_audit["FLAG_OUTLIER"])
].copy()

print("[INFO] Rows used for modeling:", len(df_model))

C:\Users\ASUS\AppData\Local\Temp\ipykernel_16312\1139542740.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_pivot[f"FLAG_OUTLIER_{col}"] = f_out
C:\Users\ASUS\AppData\Local\Temp\ipykernel_16312\1139542740.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_pivot[f"FLAG_FLAT_{col}"]    = f_flat
C:\Users\ASUS\AppData\Local\Temp\ipykernel_16312\1139542740.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider join

[INFO] Data quality summary (before filtering):
 - Flatline rows : 21587
 - Outlier rows  : 21237
 - Total rows    : 21598
[INFO] Rows used for modeling: 0


C:\Users\ASUS\AppData\Local\Temp\ipykernel_16312\1139542740.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_pivot[f"FLAG_FLAT_{col}"]    = f_flat
C:\Users\ASUS\AppData\Local\Temp\ipykernel_16312\1139542740.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_pivot[f"FLAG_OUTLIER_{col}"] = f_out
C:\Users\ASUS\AppData\Local\Temp\ipykernel_16312\1139542740.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider join

In [21]:
# ==============================
# CEK TAG YANG TIDAK ADA DATANYA (MISSING TAGS)
# ==============================
all_tags = set(df_pivot.columns)  # semua tag setelah pivot + filter
all_tags.discard("timestamp")      # hilangkan kolom timestamp jika ada
all_tags.discard(LOAD_TAG)         # hilangkan LOAD karena pasti ada

missing_tags = []

for tag in all_tags:
    # cek apakah semua nilai NaN
    if df_pivot[tag].dropna().empty:
        missing_tags.append(tag)

if missing_tags:
    print(f"[INFO] Tag yang tidak muncul sama sekali dari {START} sampai {END}:")
    for t in missing_tags:
        print(f" - {t}")
else:
    print(f"[INFO] Semua tag memiliki data minimal satu nilai dari {START} sampai {END}")

# ==============================
# OPTIONAL: simpan ke CSV
# ==============================
MISSING_TAGS_CSV = os.path.join(PLOT_ROOT, "missing_tags.csv")
if missing_tags:
    pd.DataFrame({"missing_tag": missing_tags}).to_csv(MISSING_TAGS_CSV, index=False)
    print(f"[INFO] Tag yang tidak muncul tersimpan di {MISSING_TAGS_CSV}")


[INFO] Semua tag memiliki data minimal satu nilai dari 2025-12-01 sampai 2025-12-16


In [22]:
def compute_side(df, cfg):
    tags = [t for t in cfg.get("tags", []) if t in df.columns]
    if not tags:
        return None
    return df[tags].mean(axis=1) if cfg.get("mode") == "AVG" else df[tags[0]]

In [23]:
def build_avg_target(df, name, cfg):
    vals = []
    for tags in cfg.values():
        valid = [t for t in tags if t in df.columns]
        if not valid:
            return None
        vals.append(df[valid].mean(axis=1))
    return vals[0] if len(vals) == 1 else pd.concat(vals, axis=1).mean(axis=1)


def build_delta_target(df, name, cfg):
    left = compute_side(df, cfg.get("left", {}))
    right = compute_side(df, cfg.get("right", {}))
    if left is None or right is None:
        return None
    return left - right

import numpy as np

def build_lmtd_target(df, name, cfg):
    avg_cfg = cfg.get("avg", {})

    required_keys = [
        "LMTD_Avg1",
        "LMTD_Avg2",
        "LMTD_Avg3",
        "LMTD_Avg4",
    ]

    vals = {}

    for key in required_keys:
        tags = avg_cfg.get(key, [])
        valid = [t for t in tags if t in df.columns]

        if not valid:
            print(f"[WARN] {key} kosong / tidak valid untuk {name}")
            return None

        vals[key] = df[valid].mean(axis=1)

    Th_in  = vals["LMTD_Avg1"]
    Tc_out = vals["LMTD_Avg2"]
    Th_out = vals["LMTD_Avg3"]
    Tc_in  = vals["LMTD_Avg4"]

    DT1 = Th_in - Tc_out
    DT2 = Th_out - Tc_in

    valid_mask = (DT1 > 0) & (DT2 > 0)

    lmtd = (DT1 - DT2) / np.log(DT1 / DT2)
    lmtd[~valid_mask] = np.nan
    lmtd[np.isinf(lmtd)] = np.nan

    return lmtd

In [24]:
TARGET_MAP = []

In [25]:
for group_name, group_items in groups.items():
    group_safe = safe_filename(group_name)

    for info in group_items:
        name_safe = safe_filename(info["name"])
        target_cfg = info.get("target", {})

        # ---------- DELTA ----------
        if "delta" in target_cfg:
            y_series = build_delta_target(df_pivot, info["name"], target_cfg["delta"])
            if y_series is not None:
                col_name = (
                    f"DELTA_{safe_filename(info['name'])}"
                    f"__{safe_filename(group_name)}"
                )

                df_pivot[col_name] = y_series

                TARGET_MAP.append({
                    "group": group_name,
                    "name": info["name"],
                    "col": col_name,
                    "info": {
                        **info,
                        "group_name": group_name
                    }
                })

            continue

        # ---------- LMTD ----------
        if info["name"].upper() == "LMTD":
            y_series = build_lmtd_target(df_pivot, info["name"], target_cfg)
            if y_series is not None:
                col_name = f"LMTD__{safe_filename(group_name)}"

                df_pivot[col_name] = y_series

                TARGET_MAP.append({
                    "group": group_name,
                    "name": info["name"],
                    "col": col_name,
                    "info": {
                        **info,
                        "group_name": group_name
                    }
                })

            continue

        # ---------- AVG TARGET ----------
        avg_cfg = target_cfg.get("avg", {})
        if avg_cfg:
            y_series = build_avg_target(df_pivot, info["name"], avg_cfg)
            if y_series is not None:
                col_name = f"AVG_{safe_filename(info['name'])}"
                df_pivot[col_name] = y_series
                TARGET_MAP.append({
                    "group": group_name,
                    "name": info["name"],
                    "col": col_name,
                    "info": {
                        **info,
                        "group_name": group_name
                    }
                })

            continue

        # ---------- SINGLE ----------
        for t in target_cfg.get("single", []):
            if t in df_pivot.columns:
                TARGET_MAP.append({
                    "group": group_name,
                    "name": info["name"],
                    "col": t,
                    "info": {
                        **info,
                        "group_name": group_name
                    }
                })

print(f"[INFO] Total targets: {len(TARGET_MAP)}")

[INFO] Total targets: 46


C:\Users\ASUS\AppData\Local\Temp\ipykernel_16312\2743333288.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_pivot[col_name] = y_series
C:\Users\ASUS\AppData\Local\Temp\ipykernel_16312\2743333288.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_pivot[col_name] = y_series
C:\Users\ASUS\AppData\Local\Temp\ipykernel_16312\2743333288.py:57: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at on

In [26]:
# TAGS USED IN TARGET (ANTI LEAK)
def get_tags_used_in_target(info):
    used = set()
    cfg = info.get("target", {})

    if "delta" in cfg:
        used.update(cfg["delta"].get("left", {}).get("tags", []))
        used.update(cfg["delta"].get("right", {}).get("tags", []))

    if info["name"].upper() == "LMTD":
        for tags in cfg.get("avg", {}).values():
            used.update(tags)

    used.update(cfg.get("single", []))
    return used

In [27]:
allowed_tags = set()
for group_items in groups.values():
    for item in group_items:
        # fitur single
        allowed_tags.update(item.get("features", {}).get("single", []))
        # fitur avg (jika ada)
        avg_dict = item.get("features", {}).get("avg", {})
        for v in avg_dict.values():
            allowed_tags.update(v)
        # target single
        allowed_tags.update(item.get("target", {}).get("single", []))
        # target avg
        avg_target = item.get("target", {}).get("avg", {})
        for v in avg_target.values():
            allowed_tags.update(v)
        # target delta
        delta_cfg = item.get("target", {}).get("delta", {})
        for side in ["left", "right"]:
            allowed_tags.update(delta_cfg.get(side, {}).get("tags", []))

print(f"[INFO] Total allowed tags from JSON engineer: {len(allowed_tags)}")


[INFO] Total allowed tags from JSON engineer: 97


In [28]:
X2_RANKING = {}
SUMMARY_ROWS = []

In [29]:
for t in TARGET_MAP:
    y_col = t["col"]
    info = t["info"]

    blocked = get_tags_used_in_target(info) | {y_col, LOAD_TAG}

    # Hanya tag JSON engineer + anti-leak
    x_candidates = [
        c for c in df_pivot.columns
        if c in ENGINEER_FEATURE_TAGS
        and c not in blocked
        and df_pivot[c].notna().any()
    ]

    df_corr = pd.concat([df_pivot[x_candidates], df_pivot[y_col]], axis=1).dropna()
    if df_corr.empty:
        continue

    corr = (
        df_corr[x_candidates]
        .corrwith(df_corr[y_col])
        .abs()
        .sort_values(ascending=False)
    )

    if not corr.empty:
        X2_RANKING[y_col] = corr

print(f"[INFO] X2_RANKING built for {len(X2_RANKING)} targets")

[INFO] X2_RANKING built for 34 targets


# EDA - Corellation per Target (Heatmap)

In [30]:
# HEATMAP (READ ONLY — NO RECOMPUTE)
def plot_corr_heatmap_from_ranking(target_col, save_dir, top_n=5):
    if target_col not in X2_RANKING:
        return

    corr = X2_RANKING[target_col].head(top_n)
    if corr.empty:
        return

    fig, ax = plt.subplots(figsize=(6, 0.6 * len(corr) + 1))
    ax.imshow(corr.values.reshape(-1, 1), cmap="RdBu_r", vmin=0, vmax=1)

    ax.set_yticks(range(len(corr)))
    ax.set_yticklabels(corr.index)
    ax.set_xticks([0])
    ax.set_xticklabels([target_col])
    ax.set_title(f"Top {len(corr)} Correlation vs {target_col}")

    for i, v in enumerate(corr.values):
        ax.text(
            0, i, f"{v:.3f}",
            ha="center", va="center",
            color="white" if v > 0.7 else "black"
        )

    plt.tight_layout()
    plt.savefig(
        os.path.join(save_dir, f"heatmap_corr_{safe_filename(target_col)}.jpg"),
        dpi=150
    )
    plt.close()

# RUN
print("\n=== GENERATING CORRELATION HEATMAPS (CONSISTENT) ===")

for t in TARGET_MAP:
    save_dir = os.path.join(
        PLOT_ROOT,
        safe_filename(t["group"]),
        safe_filename(t["name"])
    )
    os.makedirs(save_dir, exist_ok=True)

    plot_corr_heatmap_from_ranking(
        target_col=t["col"],
        save_dir=save_dir,
        top_n=5
    )

print("\n=== DONE ===")


=== GENERATING CORRELATION HEATMAPS (CONSISTENT) ===

=== DONE ===


# Preprocessing

In [31]:
def build_features_df(df, selected_features, load_tag=LOAD_TAG):
    """
    X1 = LOAD (wajib)
    X2..Xn = fitur hasil korelasi (tanpa LOAD)
    """
    X = pd.DataFrame(index=df.index)

    # X1: LOAD
    if load_tag in df.columns:
        X[load_tag] = df[load_tag]

    # X2+: fitur korelasi
    for f in selected_features:
        if f in df.columns and f != load_tag:
            X[f] = df[f]

    return X


In [32]:
def adjusted_r2(r2, n, k):
    """
    r2 : nilai R²
    n  : jumlah data
    k  : jumlah variabel independen (LOAD + x2 => 2)
    """
    if n <= k + 1:
        return r2  
    return 1 - (1 - r2) * (n - 1) / (n - k - 1)


In [33]:
def pseudo_r2_quantile(y_true, y_pred, q=0.1):
    """
    Hitung Koenker & Machado pseudo R² untuk quantile regression
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_q = np.quantile(y_true, q)
    
    rho = lambda u: u * (q - (u < 0))
    
    return 1 - rho(y_true - y_pred).sum() / rho(y_true - y_q).sum()

# Modeling

## Time-Series

In [34]:
def plot_timeseries_model(df_index, y_actual, y_pred_dict,
                          y_load,                      # <-- LOAD
                          target_name, feature_name, save_folder,
                          extra_text=None, mode="uni",
                          eligible_mask=None):
    """
    Time series plot dengan secondary axis (LOAD).
    """
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    import os

    os.makedirs(save_folder, exist_ok=True)

    # === Subplot dengan secondary axis ===
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # === Actual (Primary Y) ===
    fig.add_trace(
        go.Scatter(
            x=df_index,
            y=y_actual,
            mode="lines",
            name="Actual",
            line=dict(width=2, color="blue")
        ),
        secondary_y=False
    )

    # === Predictions (Primary Y) ===
    for model_name, y_pred in y_pred_dict.items():
        color = "red" if "Linear" in model_name else "green"
        fig.add_trace(
            go.Scatter(
                x=df_index,
                y=y_pred,
                mode="lines",
                name=model_name,
                line=dict(width=2, color=color)
            ),
            secondary_y=False
        )

    # === LOAD (Secondary Y) ===
    if y_load is not None:
        fig.add_trace(
            go.Scatter(
                x=df_index,
                y=y_load,
                mode="lines",
                name="Load",
                line=dict(width=2, color="orange", dash="dot")
            ),
            secondary_y=True
        )

    # === Eligible Marker (Primary Y) ===
    if eligible_mask is not None and eligible_mask.sum() > 0:
        y_offset = y_actual.max() + 0.01 * (y_actual.max() - y_actual.min())
        fig.add_trace(
            go.Scatter(
                x=df_index[eligible_mask],
                y=[y_offset] * eligible_mask.sum(),
                mode="markers",
                name="Eligible (TRUE)",
                marker=dict(color="green", size=8, symbol="triangle-up")
            ),
            secondary_y=False
        )

    # === Axis Titles ===
    fig.update_yaxes(title_text=target_name, secondary_y=False)
    fig.update_yaxes(title_text="Load", secondary_y=True)

    # === Layout ===
    fig.update_layout(
        title=f"Time Series – {target_name} (with Load)",
        xaxis_title="Time",
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.35,
            xanchor="center",
            x=0.5
        ),
        margin=dict(t=80, b=140)
    )

    # === Extra Info ===
    if extra_text:
        extra_html = extra_text.replace("\n", "<br>")
        fig.update_layout(margin=dict(t=80, b=260))
        fig.add_annotation(
            text=f"<br><b>Model Info:</b><br>{extra_html}",
            xref="paper", yref="paper",
            x=0, y=-0.25,
            showarrow=False,
            align="left",
            font=dict(size=11)
        )

    prefix = "UNI" if mode == "univariate" else "MULTI"
    safe_name = safe_filename(
        f"{prefix}_{target_name}_{feature_name}_timeseries.html"
    )

    fig.write_html(os.path.join(save_folder, safe_name))

## Regression - Univariate

In [ ]:
def run_univariate_models_and_plots(x_df, y_series, target_name, save_folder, rule_symbol=">", raw_df=None, target_info=None):

    os.makedirs(save_folder, exist_ok=True)
    safe_target = safe_filename(target_name)

    LOAD_TAG = "EWS102/10DAS0O:AI1O013305.PNT"

    # === PASTIKAN LOAD ADA DI X_DF ===
    if LOAD_TAG not in x_df.columns:
        if raw_df is not None and LOAD_TAG in raw_df.columns:
            x_df = x_df.copy()
            x_df[LOAD_TAG] = raw_df.loc[x_df.index, LOAD_TAG]
        else:
            print(f"[WARNING] LOAD tag '{LOAD_TAG}' tidak ada di X dataframe. Skip target {target_name}")
            return

    feature = LOAD_TAG
    x = x_df[[feature]].dropna()
    y = y_series.loc[x.index].dropna()
    if len(x) < 5:
        print(f"[SKIP] sample terlalu sedikit untuk {target_name}")
        return

    # === RULE & QUANTILE ===
    use_lt = rule_symbol == "<"
    q = 0.06 if use_lt else 0.94

    # === LINEAR MODELS ===
    qr_linear = QuantileRegressor(quantile=q, alpha=0).fit(x, y)
    y_pred_lr = qr_linear.predict(x)

    lr_simple = LinearRegression().fit(x, y)
    y_pred_simple = lr_simple.predict(x)

    # === ELIGIBLE (DARI LINEAR QR) ===
    mask = (y_pred_lr > y) if use_lt else (y_pred_lr < y)
    eligible_pct = (mask.sum() / len(mask)) * 100

    MIN_ELIGIBLE = 2.0
    MAX_ELIGIBLE = 20.0

    is_valid_eligible = MIN_ELIGIBLE <= eligible_pct <= MAX_ELIGIBLE
    eligible_mask = mask if is_valid_eligible else None

    # === POLYNOMIAL QR ===
    poly = PolynomialFeatures(degree=2, include_bias=False)
    x_poly = poly.fit_transform(x)
    qr_poly = QuantileRegressor(quantile=q, alpha=0).fit(x_poly, y)
    y_pred_poly = qr_poly.predict(x_poly)

    # === METRICS ===
    pseudo_r2_lr = pseudo_r2_quantile(y, y_pred_lr, q=q)
    pinball_lr = mean_pinball_loss(y, y_pred_lr, alpha=q)

    pseudo_r2_poly = pseudo_r2_quantile(y, y_pred_poly, q=q)
    pinball_poly = mean_pinball_loss(y, y_pred_poly, alpha=q)

    # === FORMULAS ===
    formula_lr = f"{target_name} = {qr_linear.intercept_:.3f} + {qr_linear.coef_[0]:.3f}*{feature}"
    a, b = qr_poly.coef_[0], qr_poly.coef_[1] if len(qr_poly.coef_) > 1 else 0
    formula_poly = f"{target_name} = {qr_poly.intercept_:.3f} + {a:.3f}*{feature} + {b:.3f}*{feature}²"

    # === PLOT 1 — LINEAR SPACE ===
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.scatter(x[feature], y, s=22, color="lightblue", label="Data")
    ax.plot(x[feature], y_pred_simple, color="gray", linewidth=2, label="Simple Linear")
    ax.plot(x[feature], y_pred_lr, color="orange", linestyle="--", linewidth=2,
            label=f"Linear Quantile (q={q})")

    ax.set_title(f"{target_name} – Linear Model")
    ax.set_xlabel(feature)
    ax.set_ylabel(target_name)
    ax.grid(True, linestyle="--", alpha=0.5)
    ax.legend()

    # ax.text(
    #     0.05, 0.95,
    #     f"{formula_lr}\n"
    #     f"R² = {pseudo_r2_lr:.3f}\n"
    #     f"Error = {pinball_lr:.3f}\n"
    #     f"Eligible = {eligible_pct:.2f}%",
    #     transform=ax.transAxes,
    #     va="top"
    # )

    ax.text(
        0.5, -0.25,  # posisi di bawah sumbu x
        f"{formula_lr}\nR² = {pseudo_r2_lr:.3f}, Pinball = {pinball_lr:.3f}, Eligible = {eligible_pct:.2f}%",
        transform=ax.transAxes,
        ha="center",
        va="top"
    )


    plt.tight_layout()
    plt.savefig(os.path.join(save_folder, f"{safe_target}_LINEAR.jpg"), dpi=150)
    plt.close()

    # === PLOT 2 — POLYNOMIAL SPACE ===
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.scatter(x[feature], y, s=22, color="lightblue", label="Data")
    ax.plot(x[feature], y_pred_poly, color="orange", linestyle="--", linewidth=2,
            label=f"Polynomial Quantile (q={q})")
    ax.plot(x[feature], y_pred_lr, color="red", linewidth=1.5, label="Linear Quantile (ref)")

    ax.set_title(f"{target_name} – Polynomial Quantile Model")
    ax.set_xlabel(feature)
    ax.set_ylabel(target_name)
    ax.grid(True, linestyle="--", alpha=0.5)
    ax.legend()

    # ax.text(
    #     0.05, 0.95,
    #     f"{formula_poly}\n"
    #     f"R² = {pseudo_r2_poly:.3f}\n"
    #     f"Error = {pinball_poly:.3f}\n"
    #     f"Eligible = {eligible_pct:.2f}%",
    #     transform=ax.transAxes,
    #     va="top"
    # )

    ax.text(
        0.5, -0.25,
        f"{formula_poly}\nR² = {pseudo_r2_poly:.3f}, Pinball = {pinball_poly:.3f}, Eligible = {eligible_pct:.2f}%",
        transform=ax.transAxes,
        ha="center",
        va="top"
    )

    plt.tight_layout()
    plt.savefig(os.path.join(save_folder, f"{safe_target}_POLY.jpg"), dpi=150)
    plt.close()

    # === TIME SERIES LINEAR QR ===
    y_pred_dict_linear = {"Linear QR": y_pred_lr}
    extra_text_linear = (
        f"Linear Quantile Regression (q={q})\n"
        f"{formula_lr}\n"
        f"Pseudo R² = {pseudo_r2_lr:.3f}\n"
        f"Eligible = {eligible_pct:.2f}%"
    )

    plot_timeseries_model(
        df_index=y.index,
        y_actual=y,
        y_pred_dict=y_pred_dict_linear,
        y_load=x_df[feature].loc[y.index],
        target_name=target_name,
        feature_name=f"{feature}_LinearQR",
        save_folder=save_folder,
        extra_text=extra_text_linear,
        mode="univariate",
        eligible_mask=eligible_mask
    )

    # === TIME SERIES POLYNOMIAL QR ===
    y_pred_dict_poly = {"Polynomial QR": y_pred_poly}
    extra_text_poly = (
        f"Polynomial Quantile Regression (q={q})\n"
        f"{formula_poly}\n"
        f"Pseudo R² = {pseudo_r2_poly:.3f}\n"
        f"Eligible = {eligible_pct:.2f}%"
    )

    plot_timeseries_model(
        df_index=y.index,
        y_actual=y,
        y_pred_dict=y_pred_dict_poly,
        y_load=x_df[feature].loc[y.index],
        target_name=target_name,
        feature_name=f"{feature}_PolyQR",
        save_folder=save_folder,
        extra_text=extra_text_poly,
        mode="univariate",
        eligible_mask=eligible_mask
    )


## Regression: Multivariate

In [ ]:
def run_multivariate_engineer(
        df_pivot,              
        x_df, y_series, target_name, save_folder,
        target_info,
        x2_ranking,
        rule_symbol=">"
):

    os.makedirs(save_folder, exist_ok=True)
    safe_target = safe_filename(target_name)

    LOAD_TAG = "EWS102/10DAS0O:AI1O013305.PNT"
    use_lt = rule_symbol == "<"
    q = 0.06 if use_lt else 0.94
    # VALIDATIO
    if LOAD_TAG not in df_pivot.columns:
        print(f"[SKIP] LOAD tidak ditemukan untuk {target_name}")
        return

    if target_name not in x2_ranking:
        print(f"[SKIP] X2 ranking tidak tersedia untuk {target_name}")
        return
    # ANTI-LEAKAGE BLOC
    block_tags = {LOAD_TAG, y_series.name} | BLOCKED_TAGS
    target_cfg = target_info.get("target", {})

    if "delta" in target_cfg:
        block_tags.update(target_cfg["delta"].get("left", {}).get("tags", []))
        block_tags.update(target_cfg["delta"].get("right", {}).get("tags", []))

    if target_info["name"].upper() == "LMTD":
        for tags in target_cfg.get("avg", {}).values():
            block_tags.update(tags)

    for t in target_cfg.get("single", []):
        block_tags.add(t)

    # >>> FIX PENTING: BLOCK AVG FEATURES <<<
    feat_cfg = target_info.get("features", {})
    for tags in feat_cfg.get("avg", {}).values():
        block_tags.update(tags)
    # SELECT X2 FROM RANKIN
    valid_x2 = [
        x for x in x2_ranking[target_name].index
        if x in ENGINEER_FEATURE_TAGS
        and x not in block_tags
        and x in df_pivot.columns
    ]

    if not valid_x2:
        print(f"[SKIP] Tidak ada X2 valid untuk {target_name}")
        return

    top_x2 = valid_x2[:2]
    best_x2 = valid_x2[0]

    summary_written = False

    # MODEL LOOP (TOP-2)
    for idx, x2_tag in enumerate(top_x2, start=1):

        df_model = pd.concat(
            [df_pivot[[LOAD_TAG, x2_tag]], y_series],
            axis=1
        ).dropna()

        if len(df_model) < 10:
            continue

        X = df_model[[LOAD_TAG, x2_tag]]
        y = df_model[target_name]

        qr = QuantileRegressor(quantile=q, alpha=0)
        qr.fit(X, y)
        y_pred = qr.predict(X)

        pseudo_r2 = pseudo_r2_quantile(y, y_pred, q=q)
        pinball = mean_pinball_loss(y, y_pred, alpha=q)

        mask = (y_pred > y) if use_lt else (y_pred < y)
        eligible_pct = (mask.sum() / len(mask)) * 100

        TARGET_ELIGIBLE = 6.0
        MIN_ELIGIBLE = 2.0
        MAX_ELIGIBLE = 20.0

        is_valid_eligible = MIN_ELIGIBLE <= eligible_pct <= MAX_ELIGIBLE
        eligible_mask = mask if is_valid_eligible else None

        a_q, b_q = qr.coef_
        c_q = qr.intercept_

        fy_expr = build_target_expression(target_info)

        formula = (
            f"{fy_expr} = {c_q:.3f} "
            f"+ {a_q:.3f}*{LOAD_TAG} "
            f"+ {b_q:.3f}*{x2_tag}"
        )

        # SAVE CSV
        rule_name = target_info.get("rule_id", target_name)
        physical_name = target_info.get("name", target_name)

        add_csv_row(rule_name, physical_name, "formula", "", formula=formula,
                    target_name=physical_name, display_name=target_info.get("group_name"), rule_symbol=rule_symbol)
        add_csv_row(rule_name, physical_name, "target", "a")
        add_csv_row(rule_name, x2_tag, "fitur", "a")
        add_csv_row(rule_name, LOAD_TAG, "general", "")

        # SUMMARY (HANYA X2 RANK #1)
        if (x2_tag == best_x2) and (not summary_written):
            corr_val = x2_ranking[target_name].loc[best_x2]

            target_label = target_info.get("rule_id", target_name)

            SUMMARY_ROWS.append({
                "target": target_label,
                "best_features": best_x2,
                "corr": float(corr_val),
                "abs_corr": abs(corr_val),
                "pseudo_r2": pseudo_r2,
                "pinball": pinball,
                "eligible_pct": eligible_pct,
                "n_samples_aligned": len(df_model)
            })

            summary_written = True

        # PLOT
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.scatter(X[LOAD_TAG], y, s=22, color="lightblue", label="Data")

        load_vals = np.linspace(X[LOAD_TAG].min(), X[LOAD_TAG].max(), 200)
        x2_fixed = X[x2_tag].mean()

        ax.plot(
            load_vals,
            c_q + a_q * load_vals + b_q * x2_fixed,
            color="orange",
            linestyle="--",
            linewidth=2.5,
            label=f"Quantile QR (q={q})"
        )

        ax.set_title(f"{target_name} – Multivariate Quantile")
        ax.set_xlabel("LOAD")
        ax.set_ylabel(target_name)
        ax.grid(True, linestyle="--", alpha=0.5)
        ax.legend()

        info = (
            f"{formula}\n"
            f"Pseudo R² = {pseudo_r2:.3f}\n"
            f"Pinball = {pinball:.3f}\n"
            f"Eligible = {eligible_pct:.2f}%"
        )

        ax.text(
            0.5, -0.18,
            info,
            transform=ax.transAxes,
            ha="center", va="top",
            bbox=dict(boxstyle="round", facecolor="white", alpha=0.8)
        )

        plt.tight_layout(rect=[0, 0.08, 1, 1])
        out_img = os.path.join(
            save_folder,
            f"{safe_target}_MULTI_{idx}_{safe_filename(x2_tag)}.jpg"
        )
        plt.savefig(out_img, dpi=150, bbox_inches="tight")
        plt.close()

        # TIME SERIES
        plot_timeseries_model(
            df_index=y.index,
            y_actual=y,
            y_pred_dict={"Multivariate QR": y_pred},
            y_load=df_model[LOAD_TAG],
            target_name=target_name,
            feature_name=f"LOAD + {x2_tag}",
            save_folder=save_folder,
            extra_text=info,
            mode="multivariate",
            eligible_mask=eligible_mask
        )

# Running Modelling

In [37]:
print("\n=== RUNNING MODELING (UNI + MULTI) ===")

for t in TARGET_MAP:
    group_safe = safe_filename(t["group"])
    name_safe  = safe_filename(t["name"])
    target_col = t["col"]
    info       = t["info"]

    save_dir = os.path.join(PLOT_ROOT, group_safe, name_safe)
    os.makedirs(save_dir, exist_ok=True)

    # === TARGET ===
    y = df_pivot[target_col].dropna()
    if y.empty:
        continue

    # === UNIVARIATE (LOAD only) ===
    X_base_uni = df_pivot[[LOAD_TAG]].loc[y.index]

    run_univariate_models_and_plots(
        x_df=X_base_uni,
        y_series=y,
        target_name=target_col,
        save_folder=save_dir,
        rule_symbol=info.get("symbol", ">"),
        raw_df=df_pivot,
        target_info=info
    )

    # === MULTIVARIATE (LOAD + top-2 X2) ===
    top_x2 = X2_RANKING.get(target_col, []).index[:2].tolist()
    X_base_multi = df_pivot[
        [LOAD_TAG] + [x for x in top_x2 if x in df_pivot.columns]
    ].loc[y.index]

    run_multivariate_engineer(
        df_pivot=df_pivot,
        x_df=X_base_multi,
        y_series=y,
        target_name=target_col,
        save_folder=save_dir,
        target_info=info,
        x2_ranking=X2_RANKING,
        rule_symbol=info.get("symbol", ">")
    )

print("\n=== ALL MODELING DONE ===")



=== RUNNING MODELING (UNI + MULTI) ===


C:\Users\ASUS\AppData\Local\Temp\ipykernel_16312\37170372.py:176: UserWarning:

Tight layout not applied. The left and right margins cannot be made large enough to accommodate all Axes decorations.

C:\Users\ASUS\AppData\Local\Temp\ipykernel_16312\37170372.py:176: UserWarning:

Tight layout not applied. The left and right margins cannot be made large enough to accommodate all Axes decorations.

C:\Users\ASUS\AppData\Local\Temp\ipykernel_16312\37170372.py:176: UserWarning:

Tight layout not applied. The left and right margins cannot be made large enough to accommodate all Axes decorations.

C:\Users\ASUS\AppData\Local\Temp\ipykernel_16312\37170372.py:176: UserWarning:

Tight layout not applied. The left and right margins cannot be made large enough to accommodate all Axes decorations.

C:\Users\ASUS\AppData\Local\Temp\ipykernel_16312\37170372.py:176: UserWarning:

Tight layout not applied. The left and right margins cannot be made large enough to accommodate all Axes decorations.

C:\Us


=== ALL MODELING DONE ===


# Save CSV Rules

In [43]:
import os
import pandas as pd
from string import ascii_lowercase

# ==============================
# PATH FILE CSV
# ==============================
PLOT_ROOT = "../plots_html_dev/UNIT1"
CSV_FORMULA_PATH = os.path.join(PLOT_ROOT, "formulas_multivariate.csv")
OUTPUT_CSV = os.path.join(PLOT_ROOT, "features_per_rule_with_load_tja1.csv")

# ==============================
# PASTIKAN DIREKTORI ADA
# ==============================
for path in [CSV_FORMULA_PATH, OUTPUT_CSV]:
    os.makedirs(os.path.dirname(path), exist_ok=True)

# ==============================
# SIMPAN CSV FORMULAS
# ==============================
def save_csv_formulas(csv_path):
    if not FORMULAS_ROWS:
        print("[INFO] Tidak ada formula untuk disimpan")
        return
    df = pd.DataFrame(FORMULAS_ROWS)
    # Pastikan drop_duplicates menyertakan kolom baru agar data tidak hilang
    df = df.drop_duplicates(subset=["formula", "tag_name", "display_name", "area_name"])
    df = df.sort_values(["area_name", "display_name"])
    df.to_csv(csv_path, index=False)
    print(f"[INFO] CSV formulas tersimpan di: {csv_path}")

save_csv_formulas(CSV_FORMULA_PATH)

# ==============================
# SIMPAN CSV FEATURES + LOAD + TOP2 ML
# ==============================
df_rows = []

LOAD_TAG = "EWS102/10DAS0O:AI1O013305.PNT"

for group_name, group_items in groups.items():
    for item in group_items:
        rule_name = f"{safe_filename(item['name'])}_{safe_filename(group_name)}"

        # Tambahkan LOAD tag
        df_rows.append({
            "rule_name": rule_name,
            "tag": LOAD_TAG,
            "f_pair": "general",
            "source": "LOAD"
        })

        # Fitur asli dari JSON
        features = item.get("features", {}).get("single", [])
        for idx, tag in enumerate(features):
            df_rows.append({
                "rule_name": rule_name,
                "tag": tag,
                "f_pair": ascii_lowercase[idx % 26],
                "source": "RECOMMENDATION"
            })

        # Fitur ML/top2 korelasi
        target_col = None
        for t in TARGET_MAP:
            if t["group"] == group_name and t["name"] == item["name"]:
                target_col = t["col"]
                break

        if target_col and target_col in X2_RANKING:
            top2_ml = X2_RANKING[target_col].index[:2].tolist()
            for idx2, tag2 in enumerate(top2_ml, start=1):
                df_rows.append({
                    "rule_name": rule_name,
                    "tag": tag2,
                    "f_pair": f"ML{idx2}",
                    "source": "ML_CORRELATION"
                })

df_out = pd.DataFrame(df_rows)
if 'source' not in df_out.columns:
    df_out['source'] = "ML"

df_out = df_out.drop_duplicates(subset=["rule_name", "tag", "source"])
df_out.to_csv(OUTPUT_CSV, index=False)
print(f"[INFO] CSV fitur + LOAD + top2 ML tersimpan: {OUTPUT_CSV}")

[INFO] CSV formulas tersimpan di: ../plots_html_dev/UNIT1\formulas_multivariate.csv
[INFO] CSV fitur + LOAD + top2 ML tersimpan: ../plots_html_dev/UNIT1\features_per_rule_with_load_tja1.csv


In [44]:
# SAVE SUMMARY CSV (FINAL)

SUMMARY_CSV_PATH = os.path.join(PLOT_ROOT, "summary_multivariate_qr.csv")

if not SUMMARY_ROWS:
    print("[INFO] Tidak ada summary untuk disimpan")
else:
    df_summary = pd.DataFrame(SUMMARY_ROWS)

    # --- pastikan tidak dobel target ---
    df_summary = df_summary.drop_duplicates(subset=["target"])

    # --- rapikan tipe data ---
    num_cols = [
        "corr", "abs_corr",
        "pseudo_r2", "pinball",
        "eligible_pct"
    ]
    for c in num_cols:
        df_summary[c] = pd.to_numeric(df_summary[c], errors="coerce")

    df_summary["n_samples_aligned"] = (
        pd.to_numeric(df_summary["n_samples_aligned"], errors="coerce")
        .astype("Int64")
    )

    # --- sorting: prioritas model yang kuat ---
    df_summary = df_summary.sort_values(
        by=["pseudo_r2", "abs_corr"],
        ascending=[False, False]
    )

    # --- simpan ---
    df_summary.to_csv(SUMMARY_CSV_PATH, index=False)

    print(f"[INFO] Summary multivariate QR tersimpan: {SUMMARY_CSV_PATH}")

[INFO] Summary multivariate QR tersimpan: ../plots_html_dev/UNIT1\summary_multivariate_qr.csv


In [45]:
# EXPORT ML TAGS DI LUAR REKOMENDASI ENGINEER (FILE BARU)

ML_OUTSIDE_PATH = os.path.join(
    PLOT_ROOT,
    "ml_tags_outside_recommendation_tja1.csv"
)

if "df_out" not in locals():
    print("[WARN] df_out tidak ditemukan, skip ML outside extraction")
else:
    rows = []

    for rule_name, g in df_out.groupby("rule_name"):

        # tag rekomendasi engineer
        rec_tags = set(
            g.loc[g["source"] == "RECOMMENDATION", "tag"]
        )

        # tag dari ML
        ml_tags = g.loc[g["source"] == "ML_CORRELATION", "tag"]

        # ML tag yang tidak ada di recommendation
        for tag in ml_tags:
            if tag not in rec_tags:
                rows.append({
                    "rule_name": rule_name,
                    "ml_tag": tag,
                    "note": "ML discovered, not in engineer recommendation"
                })

    if not rows:
        print("[INFO] Tidak ada ML tag di luar rekomendasi engineer")
    else:
        df_ml_outside = pd.DataFrame(rows)
        df_ml_outside = df_ml_outside.drop_duplicates()
        df_ml_outside.to_csv(ML_OUTSIDE_PATH, index=False)

        print(f"[INFO] ML tags outside recommendation tersimpan: {ML_OUTSIDE_PATH}")


[INFO] ML tags outside recommendation tersimpan: ../plots_html_dev/UNIT1\ml_tags_outside_recommendation_tja1.csv
